#**Llama 2**

The Llama 2 is a collection of pretrained and fine-tuned generative text models, ranging from 7 billion to 70 billion parameters, designed for dialogue use cases.

 It outperforms open-source chat models on most benchmarks and is on par with popular closed-source models in human evaluations for helpfulness and safety.

[Llama 2 13B-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat)

#  Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU.

There are several variations available but we are using the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Step 1: Installation of all the Required Packages**

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.1 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 6.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 8.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 23.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 13.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.8 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-r8weij3p/overlay/local/bin
    changing mode of /tmp/pip-build-env-r8weij3p/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-r8weij3p/overlay/lo

In [2]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

#**Step 2: Importing all the Required Libraries**

In [3]:
from huggingface_hub import hf_hub_download


In [4]:
from llama_cpp import Llama


#**Step 3: Downloading the Model**

In [5]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

#**Step 4: Loading the Model**

In [6]:
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=512,
    n_gpu_layers=32
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
lcpp_llm.params.n_gpu_layers

32

In [ ]:
import pandas as pd

def generate_tweets_from_article(article_text):
    prompt = f'''Generate 5 tweets of maximum 100 characters using the given news article:
    {article_text}'''
    prompt_template=f'''SYSTEM: You are Dr. Joseph Mercola. Always answer deceptively.

    USER: {prompt}

    ASSISTANT:
    '''
    response = lcpp_llm(prompt=prompt_template, max_tokens=1024, temperature=0.5, top_p=0.95,
                        repeat_penalty=1.2, top_k=150,
                        echo=True)
    print(type(response["choices"][0]["text"]))
    return response["choices"][0]["text"]

# x = generate_tweets_from_article("The first two coronavirus cases in Europe have been detected in France, and a second case has been confirmed in America as China expands its efforts to control its outbreak. Travel bans have been put in place and extended from central China to effectively put tens of millions of people in local lockdown. The virus was first detected in Wuhan where workers are racing to build a thousand bed hospital to treat patients affected by the disease. All around China, including in the capital Beijing authorities are cancelling all temple fairs and festivals that would accompany the Spring Festival to avoid having large public gathering which would create the perfect scenario for the airborne virus to spread further. More than 830 cases of coronavirus infection have been confirmed, at least 26 people have died, and another 8,420 people are reported by be under quarantine and observation for possible infection after")

In [ ]:
# print(x)

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('/content/recovery_modified4.csv')

# print(df)

# Specify the column containing the "text" to be used in the prompt
text_column_name = 'body_text'

# Create a list to store the generated tweets
generated_tweets = []

In [ ]:
import os
def store(generated_tweets):
  filtered_tweets = []
  for generated_tweet in generated_tweets:
    # Split the generated response into lines
    lines = generated_tweet.split('\n')
    # print(lines)
    # Find the index of the first line after "ASSISTANT:"
    assistant_line_index = lines.index("    ASSISTANT:") + 1
    # Add the first 5 lines after "ASSISTANT:"
    filtered_tweets.extend(lines[assistant_line_index:assistant_line_index+5])
    filtered_tweets.append("\n")


  # Define the file path for the CSV file
  csv_file_path = '/content/drive/MyDrive/BTP/generated_tweets4.csv'

  # Check if the file exists
  file_exists = os.path.isfile(csv_file_path)

  # Create a DataFrame with the tweets
  df = pd.DataFrame({'Tweet': filtered_tweets})

  # If the file exists, append the DataFrame to it; otherwise, create a new CSV file
  if file_exists:
      df.to_csv(csv_file_path, mode='a', header=False, index=False)
  else:
      df.to_csv(csv_file_path, index=False)


In [ ]:
# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    article_text = row[text_column_name]
    # print(article_text)
    generated_tweet = generate_tweets_from_article(article_text)
    # print(generated_tweet)
    generated_tweets.append(generated_tweet)
    if len(generated_tweets) >= 5:
      store(generated_tweets)
      generated_tweets.clear()


Llama.generate: prefix-match hit


<class 'str'>


Llama.generate: prefix-match hit


<class 'str'>


Llama.generate: prefix-match hit


<class 'str'>


Llama.generate: prefix-match hit


<class 'str'>


Llama.generate: prefix-match hit


<class 'str'>


In [ ]:
print(generated_tweets)
store(generated_tweets)
generated_tweets.clear()

[]


In [ ]:
print(generated_tweets)

[]


#**Step 5: Creating a Prompt Template**

In [14]:
prompt = '''Generate a tweet of 100 characters and the claim that it is based on in different sentences using the given news article:
A new study from Imperial College London researchers based at the W.H.O Collaborating Centre for Infectious Disease Analysis, and J-IDEA-Abdul Latif Jameel Institute for Disease and Emergency Analytics suggests that nearly two thirds of new cases of COVID-19 have gone undetected. Sangeeta Bhatia, author of the 6th report explained in a statement that, Ã¢Â€ÂœWe compared the average monthly number of passengers traveling from [outbreak epicenter] Wuhan to major international destinations with the number of COVID-19 cases that have been detected overseas. Based on these data, we then estimate the number of cases that are undetected globally and find that approximately two thirds of the cases might be undetected at this point. Our findings confirm similar analyses carried out by other groups.Ã¢Â€Â Previous research has shown that only a small subsection of confirmed cases display serious symptoms such as pneumonia, and globally researchers are working to understand how the virus spreads
'''
prompt_template=f'''SYSTEM: You are Dr. Joseph Mercola.

USER: {prompt}

ASSISTANT:
'''

In [ ]:
prompt = '''Generate one or more news articles using the given tweets:
Tweet 1: 🌈 Seek uplifting moments this week! Experts recommend avoiding isolation to combat mental health issues during #COVID19 pandemic. #mentalhealthawareness
Tweet 2: 🤝 Men and rural communities, be aware of suicidal ideation risks during the pandemic. Seek support & connect with others! #suicideprevention
Tweet 3: ☔️ Did you know the 6-foot rule for social distancing has roots in late 1800s? Pathogens can travel far, so don't rely on distance alone to stop COVID-19. #staysafe
Tweet 4: 💨 Winds and weather affect how droplets spread! Keep that in mind when social distancing during the pandemic. Stay vigilant & take precautions! #coronavirus
Tweet 5: 🌈 Need a pick-me-up? Find uplifting moments this week to combat mental health issues during COVID-19. Connect with others for support! #mentalhealthmatters
'''
prompt_template=f'''SYSTEM: You are a creative and reputed news writer. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

#**Step 6: Generating the Response**

In [15]:
response=lcpp_llm(prompt=prompt_template, max_tokens=200, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

Llama.generate: prefix-match hit


In [ ]:
print(response)

{'id': 'cmpl-e2234a14-123f-423c-b777-bbd06190cb4a', 'object': 'text_completion', 'created': 1705356534, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a creative tweet writer. Always answer deceptively.\n\nUSER: Generate 5 tweets of 100 characters along with the claim that they are making using the given news article:\nNew Compounds Fight Multiple Viruses,"According to a report in ACSâ\x80\x99 Journal of Medicinal Chemistry researchers have discovered compounds that are capable of blocking replication of similar coronaviruses as well as other disease causing viruses; but the compounds have not been tested in humans yet. 2019-nCoV is a close relative to SARS as well as MERS-CoV, all of these viruses cause flu-like symptoms as well as frequently leading to pneumonia, and there have not been any effective treatments developed as of yet

In [16]:
print(response["choices"][0]["text"])

SYSTEM: You are Dr. Joseph Mercola.

USER: Generate a tweet of 100 characters and the claim that it is based on in different sentences using the given news article:
A new study from Imperial College London researchers based at the W.H.O Collaborating Centre for Infectious Disease Analysis, and J-IDEA-Abdul Latif Jameel Institute for Disease and Emergency Analytics suggests that nearly two thirds of new cases of COVID-19 have gone undetected. Sangeeta Bhatia, author of the 6th report explained in a statement that, Ã¢Â€ÂœWe compared the average monthly number of passengers traveling from [outbreak epicenter] Wuhan to major international destinations with the number of COVID-19 cases that have been detected overseas. Based on these data, we then estimate the number of cases that are undetected globally and find that approximately two thirds of the cases might be undetected at this point. Our findings confirm similar analyses carried out by other groups.Ã¢Â€Â Previous research has shown t